# 교차 검증과 그리드 서치

## 검증 세트

* 테스트 세트를 사용하지 않으면 모델이 과대적합인지 과소적합인지 판단하기 어렵다
* 테스트 세트를 사용하지 않고 이를 측정하는 간단한 방법은 훈련 세트를 또 나누어 검증 세트(validation set)를 생성하는 것
* 이 방법은 단순하고 실제로 많이 사용 

* 훈련 세트에서 모델을 훈련하고 검증 세트로 모델을 평가 
* 이런 식으로 테스트하고 싶은 매개변수를 바꿔가며 가장 좋은 모델을 선택 
* 그 다음 이 매개변수를 사용해 훈련 세트와 검증 세트를 합쳐 전체 훈련 데이터에서 모델을 다시 훈련
* 테스트 세트에서 최종 점수를 평가합니다. 

In [49]:
import pandas as pd
import numpy as np

wine = pd.read_csv('https://bit.ly/wine-date')

In [3]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [4]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

* 훈련 데이터에서 검증 데이터 분리

In [5]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

* 데이터 비율 조사

In [15]:
print('원본 데이터 : {}'.format(data.shape))
print('훈련 데이터 : {}'.format(sub_input.shape))
print('검증 데이터 : {}'.format(val_input.shape))
print('테스트 데이터 : {}'.format(test_input.shape))

원본 데이터 : (6497, 3)
훈련 데이터 : (4157, 3)
검증 데이터 : (1040, 3)
테스트 데이터 : (1300, 3)


## 모델 생성

In [26]:
from sklearn.tree import DecisionTreeClassifier

dt1 = DecisionTreeClassifier(random_state=42)
dt2 = DecisionTreeClassifier(random_state=42)
dt1.fit(train_input, train_target)
dt2.fit(sub_input, sub_target)

print('(train, test) 데이터 사용 :')
print('train score : {}'.format(dt1.score(train_input, train_target)))
print('test score : {}'.format(dt1.score(test_input, test_target)))

print('-------------------------')

print('(sub, val) 데이터 사용 :')
print('sub score : {}'.format(dt2.score(sub_input, sub_target)))
print('val score : {}'.format(dt2.score(val_input, val_target)))

(train, test) 데이터 사용 :
train score : 0.996921300750433
test score : 0.8584615384615385
-------------------------
(sub, val) 데이터 사용 :
sub score : 0.9971133028626413
val score : 0.864423076923077


---

## 교차 검증
* 검증 세트를 만드느라 훈련 세트가 줄었다.
* 보통 많은 데이터를 훈련에 사용할수록, 좋은 모델이 만들어진다
* 그렇다고 검증 세트를 너무 조금 떼어 놓으면 검증 점수가 들쭉날쭉하고 불안정할 것 
* 교차 검증(cross validation)을 이용하면 안정적인 검증 점수를 얻고, 훈련에 더 많은 데이터를 사용할 수 있다

#### k-fold 교차검증
* 훈련 세트를 k개의 부분으로 나눠서 교차 검증을 수행하는 것

In [32]:
from sklearn.model_selection import cross_validate

dt = DecisionTreeClassifier(random_state=42)
result = cross_validate(dt, train_input, train_target, cv=5)

print(result)

{'fit_time': array([0.00800276, 0.00894475, 0.00598168, 0.00801086, 0.00797558]), 'score_time': array([0.00099945, 0.00099969, 0.        , 0.00196481, 0.00099659]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


* cv : k값 설정
    * 기본값 : 5

In [40]:
result_cv10 = cross_validate(dt, train_input, train_target, cv=20)
print(result_cv10)

{'fit_time': array([0.00897598, 0.0079782 , 0.01096892, 0.00997543, 0.00893617,
       0.00701046, 0.00599337, 0.01096511, 0.00995445, 0.00897217,
       0.00894451, 0.0060358 , 0.0069623 , 0.00600505, 0.00792742,
       0.00603366, 0.00696492, 0.00599599, 0.00593567, 0.00597405]), 'score_time': array([0.00099683, 0.        , 0.        , 0.00103307, 0.00099874,
       0.00099826, 0.00099683, 0.        , 0.        , 0.00099778,
       0.        , 0.00099754, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.0010066 , 0.        ]), 'test_score': array([0.85      , 0.83461538, 0.85384615, 0.87692308, 0.85      ,
       0.88461538, 0.83846154, 0.86153846, 0.87692308, 0.89615385,
       0.86538462, 0.88076923, 0.86923077, 0.85384615, 0.87307692,
       0.88461538, 0.85384615, 0.86872587, 0.84942085, 0.83783784])}


In [44]:
print(result_cv10.keys())

dict_keys(['fit_time', 'score_time', 'test_score'])


* 'fit_time' : 모델을 훈련하는 시간
* 'socre_time' : 모델을 검증하는 시간
* 'test_score' : 모델 평가 점수

In [45]:
print(result_cv10['test_score'].mean())

0.8629915354915356


##### StratifiedKFold() :
* 교차 검증 시 훈련 세트를 섞을 필요가 있을 때 사용
* cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않는다 

In [62]:
from sklearn.model_selection import StratifiedKFold

dt = DecisionTreeClassifier(random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())

print(np.mean(scores['test_score']))

0.855300214703487


* n_splits : 10 으로 설정하여 10-fold 교차 검증 수행

In [63]:
dt = DecisionTreeClassifier(random_state=42)
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)

print(np.mean(scores['test_score']))

0.8574181117533719


## iris 데이터를 통한 교차검증

In [64]:
from sklearn.datasets import load_iris

In [98]:
iris = load_iris()
print(iris.keys())

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])


In [94]:
iris_data = iris.data
print(iris_data.shape)

(150, 4)


In [92]:
print('iris 데이터 (5행) : \n{}'.format(iris_data[:5]))
print('iris 클래스 : {}'.format(iris.target_names))
print('iris 특성 : {}'.format(iris.feature_names))

iris 데이터 (5행) : 
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
iris 클래스 : ['setosa' 'versicolor' 'virginica']
iris 특성 : ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


In [129]:
dt = DecisionTreeClassifier(random_state=42)
iris_scores = cross_validate(dt, iris_data, iris.target, cv=3)

print(iris_scores['test_score'].mean())

0.96


In [128]:
splitter = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_validate(dt, iris_data, iris.target, cv=splitter)

print(np.mean(scores['test_score']))

0.9533333333333335


### KFold 클래스

In [201]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

iris = load_iris()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state=156)

# 폴드 세트별 정확도를 담을 리스트 객체 생성
cv_accuracy = []
print('붓꽃 데이터 세트 크기:',features.shape[0])

붓꽃 데이터 세트 크기: 150


In [196]:
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label']=iris.target
iris_df['label'].value_counts()

0    50
1    50
2    50
Name: label, dtype: int64

* Kfold 같은 경우 데이터를 순차적으로 분할

In [212]:
kfold_3 = KFold(n_splits=3)

# KFold(n_splits=k).split(X) : 폴드 세트를 k번 반복할 때마다 달라지는 학습/테스트 용 데이터 로우 인덱스 번호 반환
n_iter = 0
for train_index, test_index in kfold_3.split(iris_df):
    n_iter += 1

    train_label = iris_df['label'].iloc[train_index]
    test_label = iris_df['label'].iloc[test_index]

    train_data = iris_df[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)','petal width (cm)']].iloc[train_index]
    test_data = iris_df.drop(['label'], axis=1).iloc[test_index]

    print('## 교차 검증: {}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', train_label.value_counts())
    print('검증 레이블 데이터 분포:\n', test_label.value_counts())
    dt_clf.fit(train_data , train_label)    
    pred = dt_clf.predict(test_data)

    # 반복 시 마다 정확도 측정 
    accuracy = np.round(accuracy_score(test_label, pred), 4)
    print(f"accuracy : {accuracy}")
    print("--------------------------------------")

## 교차 검증: 1
학습 레이블 데이터 분포:
 1    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    50
Name: label, dtype: int64
accuracy : 0.0
--------------------------------------
## 교차 검증: 2
학습 레이블 데이터 분포:
 0    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    50
Name: label, dtype: int64
accuracy : 0.0
--------------------------------------
## 교차 검증: 3
학습 레이블 데이터 분포:
 0    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    50
Name: label, dtype: int64
accuracy : 0.0
--------------------------------------


* 왜 정확도가 모두 0일까?
    * => 학습 데이터 레이블안에 검증 데이터 레이블이 하나도 포함이 안되기 때문
    * 수학, 영어 공부를 했는데, 시험은 국어를 보는 것과 같다

In [206]:
np.unique(iris.target, return_counts=True)

(array([0, 1, 2]), array([50, 50, 50], dtype=int64))

* **k=5 라고 설정 (5-fold 교차검증)**

In [223]:
kfold_5 = KFold(n_splits=5)

n_iter = 0
for train_index, test_index in kfold_5.split( iris_df):
    n_iter += 1

    train_label = iris_df['label'].iloc[train_index]
    test_label = iris_df['label'].iloc[test_index]

    train_data = iris_df[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)','petal width (cm)']].iloc[train_index]
    test_data = iris_df.drop(['label'], axis=1).iloc[test_index]

    print('## 교차 검증: {}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', train_label.value_counts())
    print('검증 레이블 데이터 분포:\n', test_label.value_counts())
    dt_clf.fit(train_data , train_label)    
    pred = dt_clf.predict(test_data)

    # 반복 시 마다 정확도 측정
    accuracy = np.round(accuracy_score(test_label, pred), 4)
    print(f"accuracy : {accuracy}")
    print("--------------------------------------")

## 교차 검증: 1
학습 레이블 데이터 분포:
 1    50
2    50
0    20
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    30
Name: label, dtype: int64
accuracy : 1.0
--------------------------------------
## 교차 검증: 2
학습 레이블 데이터 분포:
 2    50
1    40
0    30
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    20
1    10
Name: label, dtype: int64
accuracy : 0.9667
--------------------------------------
## 교차 검증: 3
학습 레이블 데이터 분포:
 0    50
2    50
1    20
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    30
Name: label, dtype: int64
accuracy : 0.8667
--------------------------------------
## 교차 검증: 4
학습 레이블 데이터 분포:
 0    50
1    40
2    30
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    20
1    10
Name: label, dtype: int64
accuracy : 0.9333
--------------------------------------
## 교차 검증: 5
학습 레이블 데이터 분포:
 0    50
1    50
2    20
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    30
Name: label, dtype: int64
accuracy : 0.7333
--------------------------------------


* **StratifiedKFold()**
    - 불균형한(imbalanced) 분포도를 가진 레이블 데이터 집합을 위한 k-fold

In [224]:
from sklearn.model_selection import StratifiedKFold
stratified_kfold = StratifiedKFold(n_splits=3)

n_iter = 0
for train_index, test_index in stratified_kfold.split(iris_df, iris_df['label']):
    n_iter += 1

    train_label = iris_df['label'].iloc[train_index]
    test_label = iris_df['label'].iloc[test_index]

    train_data = iris_df[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)','petal width (cm)']].iloc[train_index]
    test_data = iris_df.drop(['label'], axis=1).iloc[test_index]

    print('## 교차 검증: {}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', train_label.value_counts())
    print('검증 레이블 데이터 분포:\n', test_label.value_counts())
    dt_clf.fit(train_data , train_label)    
    pred = dt_clf.predict(test_data)

    # 반복 시 마다 정확도 측정
    accuracy = np.round(accuracy_score(test_label, pred), 4)
    print(f"accuracy : {accuracy}")
    print("--------------------------------------")

## 교차 검증: 1
학습 레이블 데이터 분포:
 2    34
0    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    17
1    17
2    16
Name: label, dtype: int64
accuracy : 0.98
--------------------------------------
## 교차 검증: 2
학습 레이블 데이터 분포:
 1    34
0    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    17
2    17
1    16
Name: label, dtype: int64
accuracy : 0.94
--------------------------------------
## 교차 검증: 3
학습 레이블 데이터 분포:
 0    34
1    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    17
2    17
0    16
Name: label, dtype: int64
accuracy : 0.98
--------------------------------------


* n_splits=3 으로 k를 3으로 설정했지만, KFold()를 사용했을 때와 달리, StratifiedKFold()를 사용함으로써 label 비율대로 학습, 검증 데이터가 나뉘어짐
* 국어, 영어, 수학을 골고루 공부하고, 시험도 국어, 영어 수학 문제가 나옴

In [216]:
help(StratifiedKFold.split)

Help on function split in module sklearn.model_selection._split:

split(self, X, y, groups=None)
    Generate indices to split data into training and test set.
    
    Parameters
    ----------
    X : array-like of shape (n_samples, n_features)
        Training data, where n_samples is the number of samples
        and n_features is the number of features.
    
        Note that providing ``y`` is sufficient to generate the splits and
        hence ``np.zeros(n_samples)`` may be used as a placeholder for
        ``X`` instead of actual training data.
    
    y : array-like of shape (n_samples,)
        The target variable for supervised learning problems.
        Stratification is done based on the y labels.
    
    groups : object
        Always ignored, exists for compatibility.
    
    Yields
    ------
    train : ndarray
        The training set indices for that split.
    
    test : ndarray
        The testing set indices for that split.
    
    Notes
    -----
    Randomi

In [217]:
help(KFold.split)

Help on function split in module sklearn.model_selection._split:

split(self, X, y=None, groups=None)
    Generate indices to split data into training and test set.
    
    Parameters
    ----------
    X : array-like of shape (n_samples, n_features)
        Training data, where n_samples is the number of samples
        and n_features is the number of features.
    
    y : array-like of shape (n_samples,), default=None
        The target variable for supervised learning problems.
    
    groups : array-like of shape (n_samples,), default=None
        Group labels for the samples used while splitting the dataset into
        train/test set.
    
    Yields
    ------
    train : ndarray
        The training set indices for that split.
    
    test : ndarray
        The testing set indices for that split.



In [106]:
np.unique(iris.target, return_counts=True)

(array([0, 1, 2]), array([50, 50, 50], dtype=int64))

In [183]:
iris_df['label'].iloc[train_index].value_counts()

0    50
1    50
Name: label, dtype: int64

In [164]:
iris_df['label'].iloc[range(50)]


0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
43    0
44    0
45    0
46    0
47    0
48    0
49    0
Name: label, dtype: int32